PROTECTED AREAS PREPROCESSING
takes as input the polygons (and multipolygons) corresponding to protected areas.
outputs a boolean 2d array 
PB: 200 000 polygons, each defined by ~1000 points ... !! Too big to handle ... !

In [1]:
import geopandas as gpd
import numpy as np
import matplotlib as plt
import pandas as pd

In [ ]:
protected_areas_polygons = pd.read_file('/home/joon/data/protected-areas/WDPA_Jan2020-shapefile-polygons.shp')

In [2]:
protected_areas_polygons = gpd.read_file('/home/joon/data/protected-areas/WDPA_Jan2020-shapefile-polygons.shp').geometry

In [81]:
foo = gpd.read_file('/home/joon/data/protected-areas/WDPA_Jan2020-shapefile-points.shp').geometry

In [71]:
polygons_coords = []
for poly in protected_areas_polygons:
    if poly.type == 'Polygon':
        polygons_coords.append(list(poly.exterior.coords))
    elif poly.type == 'MultiPolygon':
        for p in list(poly):
            polygons_coords.append(list(p.exterior.coords))

In [ ]:
param = 500
nlon, nlat = 9*np.array([2,1])*param+1
lon, lat = np.meshgrid(np.linspace(-180,180,num=nlon), np.linspace(-90,90,num=nlat))
points = np.vstack((lon.flatten(),lat.flatten())).T

In [151]:
# polygon has to be given as [(1,1), (5,1), (5,9),(3,2),(1,1)] 
def polygon_to_boolean_array(polygon):
    polygon = np.array(polygon)
    approx_polygon = plt.path.Path(np.multiply(np.rint(np.divide(polygon+np.array([180,90]),np.array([360/(nlon-1),180/(nlat-1)]))),np.array([360/(nlon-1),180/(nlat-1)]))-np.array([180,90]))

    boolean_mask = approx_polygon.contains_points(points,transform=True)
    contained_points = points.reshape((nlon,nlat))[boolean_mask]
    return contained_points

In [ ]:
from multiprocessing import Pool
with Pool(processes=11) as pool:
    pool.map(??, polygons_coords)